In [1]:

from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.utils import plot_model
import tensorflow as tf
from imageio import imread
import numpy as np
import cv2
import datetime
import tkinter

from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetections2 import DecodeDetections2
from keras_layers.keras_layer_L2Normalization import L2Normalization


from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms


# Set the image size.
img_height = 300
img_width = 300

# 1: Build the Keras model
timea = datetime.datetime.now()
K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, 3),
                n_classes=4,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], # The scales for MS COCO are [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
                two_boxes_for_ar1=True,
                steps=[8, 16, 32, 64, 100, 300],
                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                clip_boxes=False,
                variances=[0.1, 0.1, 0.2, 0.2],
                coords='centroids',
                normalize_coords=True,
                subtract_mean=[123, 117, 104],
                swap_channels=[2, 1, 0],
                confidence_thresh=0.252,
                iou_threshold=0.45,
                top_k=200,
                nms_max_output_size=400)

# 2: Load the trained weights into the model.

#  Set the path of the trained weights.

weights_path = 'C:/Users/Alok Singh/Desktop/Face detection and recognition/Face recognition trained weights.h5'

model.load_weights(weights_path, by_name=True)

# 3: Compile the model so that Keras won't complain the next time you load it.

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=5e-04)

ssd_loss = SSDLoss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)
timeb = datetime.datetime.now()
print(timeb-timea)

# open webcam
cap = cv2.VideoCapture(0)

while(cap.isOpened()):
    timea = datetime.datetime.now()
    orig_images = [] # Store the images here.
    input_images = [] # Store resized versions of the images here.
    ret, frame = cap.read()
    print(frame.shape)
    orig_images.append(frame)
    img=cv2.resize(frame,dsize=(300,300))
    img = image.img_to_array(img) 
    input_images.append(img)
    input_images = np.array(input_images)
    y_pred = model.predict(input_images)


    confidence_threshold = 0.1

    y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]

    np.set_printoptions(precision=2, suppress=True, linewidth=90)
    print("Predicted boxes:\n")
    print('   class   conf xmin   ymin   xmax   ymax')
    print(y_pred_thresh[0])
    

# 4. Visualize the predictions

#We just resized the input image above and made predictions on the distorted image. We'd like to visualize the predictions on the image in its original size though, so below we'll transform the coordinates of the predicted boxes accordingly.

# Display the image and draw the predicted boxes onto it.

# Set the name of the classes, in case of pedestrian uncomment the 'pedestrian' class name below and comment the upper classes name

    
    classes = ['background','alok','harsh','rahul','varadaraja']
   #classes = ['background','pedestrian']
    
    
    frameHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    for box in y_pred_thresh[0]:
        # Transform the predicted bounding boxes for the 300x300 image to the original image dimensions.
        xmin = box[2] * orig_images[0].shape[1] / img_width
        ymin = box[3] * orig_images[0].shape[0] / img_height
        xmax = box[4] * orig_images[0].shape[1] / img_width
        ymax = box[5] * orig_images[0].shape[0] / img_height
        

        cv2.rectangle(frameHSV,(int(xmin),int(ymin)),(int(xmin)+100,int(ymin)+30),(12*int(box[0]),100,100),2)
        cv2.putText(frameHSV,classes[int(box[0])]+str(box[1]),(int(xmin)+1,int(ymin)+25), 2, 1,(255,255,255),2)
        cv2.rectangle(frameHSV,(int(xmin),int(ymin)),(int(xmax),int(ymax)),(12*int(box[0]),100,100),2)
    print (frameHSV.shape)
    frame = cv2.cvtColor(frameHSV, cv2.COLOR_HSV2BGR)
    cv2.imshow('video',frame)
    timeb = datetime.datetime.now()
    print(timeb-timea)
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break;
cap.release()
cv2.destroyAllWindows()



C:\Users\Alok Singh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


0:00:05.500866
(480, 640, 3)
Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
[]
(480, 640, 3)
0:00:03.322038
(480, 640, 3)
Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
[]
(480, 640, 3)
0:00:00.063241
(480, 640, 3)
Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
[]
(480, 640, 3)
0:00:00.083788
(480, 640, 3)
Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
[]
(480, 640, 3)
0:00:00.077060
(480, 640, 3)
Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
[]
(480, 640, 3)
0:00:00.071509
(480, 640, 3)
Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
[]
(480, 640, 3)
0:00:00.072457
(480, 640, 3)
Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
[]
(480, 640, 3)
0:00:00.064977
(480, 640, 3)
Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
[]
(480, 640, 3)
0:00:00.077838
(480, 640, 3)
Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
[]
(480, 640, 3)
0:00:00.070883
(480, 640, 3)
Predicted boxes:
